In [8]:
import requests
import json

In [9]:
def get_data(uid):
    '''进入登陆状态的session'''
    login_url='http://localhost:3000/login/cellphone?phone=13408462303&password=shiyan123456789'
    s=requests.Session()
    responsesdata1=s.get(url=login_url)
    uid=uid

    recharge_url='http://localhost:3000/user/playlist?uid='+str(uid)
    responsesdata2=s.get(url=recharge_url)
    result=json.loads(responsesdata2.text)
    if(result['code']==200):
        with open(str('用户歌单/')+str(uid)+".json", "w",encoding='utf-8') as f:
            f.write(json.dumps(result, ensure_ascii=False, indent=4, separators=(',', ':')))
    if(result['code']!=200):
        return 0

    with open('用户歌单/'+str(uid)+'.json','r',encoding='utf8')as fp:
        t_data = json.load(fp)
    temp_list_ids=[]
    if(len(t_data['playlist'])==0):
        return 0

    recharge_url='http://localhost:3000/user/detail?uid='+str(uid)
    responsesdata2=s.get(url=recharge_url)
    result=json.loads(responsesdata2.text)
    if(result['code']==200):
        with open(str('用户详情/')+str(uid)+".json", "w",encoding='utf-8') as f:
            f.write(json.dumps(result, ensure_ascii=False, indent=4, separators=(',', ':')))


    recharge_url='http://localhost:3000/user/follows?uid='+str(uid)
    responsesdata2=s.get(url=recharge_url)
    result=json.loads(responsesdata2.text)
    if(result['code']==200):
        with open(str('用户关注/')+str(uid)+".json", "w",encoding='utf-8') as f:
            f.write(json.dumps(result, ensure_ascii=False, indent=4, separators=(',', ':')))



In [10]:
import os
import requests
import re
import pandas as pd
def download_songs(list_id,uid):
    headers={
    'Referer':'https://music.163.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.89 Safari/537.36'
    }

    list_id = list_id
    list_url = 'https://music.163.com/playlist?id=%s' % list_id
    res = requests.get(list_url, headers=headers)
    list_title = re.findall(r'<title>(.*?)</title>',res.text)[0][:-13].replace('/', '_')
    song_list_folder_path =str(list_id)
    all_folder_path=str(uid)
    if not os.path.exists(all_folder_path):
        os.mkdir(all_folder_path)

    song_count = 0
    song_ids = re.findall(r'<a href="/song\?id=(\d+)">(.*?)</a>', res.text)
    song_id=[]
    song_name=[]
    for i in song_ids:
        song_id.append(i[0])
        song_name.append(i[1])
    temp_data=pd.DataFrame({'song_id':song_id,'song_name':song_name})
    temp_data.to_csv(all_folder_path+'/'+song_list_folder_path+'.csv',encoding='utf-8',index=False)
    # for song_id in song_ids:
    #     download_url = "http://music.163.com/song/media/outer/url?id=%s" % song_id[0]
    #     try:
    #         with open(folder_path + "/" + song_id[1] +'.mp3', 'wb') as f:
    #             song_count += 1
    #             f.write(requests.get(download_url, headers=headers).content)
    #     except FileNotFoundError:
    #         print("%s 下载失败" % song_id[1])
    #         pass
    #     except OSError:
    #         print("%s 下载失败" % song_id[1])
    #         pass
    # print("总共%d首歌，下载完毕！" % song_count)


In [11]:
import shutil
def get_user_songlist(uid):
    get_data(uid)
    with open('用户歌单/'+str(uid)+'.json','r',encoding='utf8')as fp:
        json_data = json.load(fp)
    temp_list_ids=[]
    if(len(json_data['playlist'])==0):
        return 0
    for i in range(len(json_data['playlist'])):
        temp_list_ids.append(json_data['playlist'][i]['id'])
        download_songs(json_data['playlist'][i]['id'],uid)
    shutil.move(str(uid),'歌单存储')

In [12]:
# uid=1776129476
# get_user_songlist(uid)

In [13]:
import pandas as pd
ids=pd.read_csv('total_user_id.csv',encoding='utf-8',converters={'ID':str})['ID'].tolist()

In [14]:
for i in range(6050+250,len(ids)):
    get_user_songlist(ids[i])
    if i % 100==0:
        print(i)


6300
6400
6500


KeyboardInterrupt: 